# Deep Learning - MCH1
Fachdozent: Martin Melchior     
Student: Manuel Schwarz   
HS23

Dieses Notebook bearbeitet die Mini-Challenge 1 des Moduls Deep Learning (del).   
Die Performance der Modelle wurde mit **wandb.ai** aufgezeichnet und kann [hier](https://wandb.ai/manuel-schwarz/del-mc1/workspace?workspace=user-manuel-schwarz) eingesehen werden.  

<div class="alert alert-block alert-info">
<b>Aufgabenstellung:</b> Eine Blaue Box beschreibt die Aufgabe aus der Aufgabenstellung 'SGDS_DEL_MC1.pdf' 
</div>

<div class="alert alert-block alert-success">
<b>Antworte:</b> Eine Grüne Box beschreibt die Bearbeitung / Reflektion der Aufgabenstellung
</div>

In [ ]:
import os
import copy
import time
import torch
import wandb
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm 
from datetime import datetime
from torch.optim import lr_scheduler
from sklearn.model_selection import KFold
from torchvision import datasets, models, transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device: ', device)

# sound
import time
import winsound
import datetime

# Schritt 1: Auswahl Task / Datensatz

<div class="alert alert-block alert-info">
    
1. Mache Dir Gedanken, mit welchen Daten Du arbeiten möchtest und welcher Task gelernt werden soll.
    
2. Diskutiere die Idee mit dem Fachcoach.
    
</div>


### Daten und Task
Pytorch stellt einige Datensets zur Verfügung [datasets torch](https://pytorch.org/vision/main/datasets.html).
Verschiedene Kategorien stehen zur Auswahl:
- Image classification
- Image detection or segmentation
- Optical Flow
- Stereo Matching
- Image pairs
- Image captioning
- video classification
- Base classes for custom datasets


**Datenset**  
Eine beliebtes Datenset ist CIFAR10. Es beinhaltet Bilder von 10 Klassen (Flugzeuge, Katzen, Vögel, etc.), die Bilder kommen mit einer Auflösung von 32x32x3 pixel (rgb). Viele Tutorials starten mit diesem Datenset, das lässt darauf schliessen, dass der Rechenaufwand für die Hardware in einem vernünpftigen Rahmen liegt. Daher wird CIFAR10 als Datenset für diese Challenge verwendet.

**Task**  
Anhand von CIFAR10 soll ein Modell erstellt werden, welches die Klasse eines Bildes korrekt klassifiziert.



![](cifar10.png)

# Schritt 2: Daten Kennenlernen

<div class="alert alert-block alert-info">
    
1. Mache Dich mit dem Datensatz vertraut, indem Du eine explorative Analyse der Features durchführst: z.B. Vergleich der Klassen pro Feature, Balanciertheit der Klassen. 
2. Führe ein geeignetes Preprocessing durch, z.B. Normalisierung der Daten.  
    
</div>

### 1. Explorative Datenanalyse
[Tutorial Dataloader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)

In [ ]:
# Laden der Daten von CIFAR10
data_path = './data/'
train_data = torchvision.datasets.CIFAR10(data_path, train=True, download=True)

test_data = torchvision.datasets.CIFAR10(data_path, train=False, download=True)

Datengrösse der Trainings- und Testdaten

In [ ]:
print(f'Anzahl Trainingsdaten: {len(train_data)}\n'
      f'Anzahl Testdaten: {len(test_data)}')

**Wie sind die Bilder im Datensatz gespeichert?**  
Die Bilder sind direkt auf dem Datenset via dem Index abrufbar. Ein Tupel mit dem Bild (RGB, 32x32 pixel) und dem Label (6).

In [ ]:
img_0, label_0 = train_data[0]
img_0, label_0

In [ ]:
# deffinieren der Labels CIFAR10
labels_cifar10_dict = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck",
}
labels_cifar10_list = list(labels_cifar10_dict.values())

**Visualisierung der Bilder**  
Die Bilder können mit matplotlib und imshow() direkt vom Dateset über den index dargestellt werden.


In [ ]:
figure = plt.figure(figsize=(6,6))
cols, rows = 5, 5

for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_data), size=(1,)).item()
    img, label = train_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_cifar10_list[label], fontsize=8)
    plt.axis('off')
    plt.imshow(img)
plt.show()

**Untersuchen der Verteilungen der Klassen**  
Folgend werden die Verteilungen der Klassen der Cifar10 Datensets auf den Trainings- und Testdaten geprüft.

In [ ]:
train_target = train_data.targets
test_target = test_data.targets
print(f'Labels in Trainingsdaten: {len(train_target)}')
print(f'Labels in Testdaten: {len(test_target)}')

In [ ]:
figure, ax = plt.subplots(1,2, figsize=(10, 3))

ax[0].hist(train_target)
ax[0].set_xticks(np.arange(10))
ax[0].set_xticklabels(labels_cifar10_list, rotation=45)
ax[0].set_title('Trainingset', fontsize=8)

ax[1].hist(test_target)
ax[1].set_xticks(np.arange(10))
ax[1].set_xticklabels(labels_cifar10_list, rotation=45)
ax[1].set_title('Testset', fontsize=8)

plt.suptitle('Verteilung der Klassen von Cifar-10', fontsize=12)
plt.show()

<div class="alert alert-block alert-success">
    
Das Trainingsset umfasst total 50'000 Bilder, davon sind jeweils 5000 Bilder jeder Klasse enthalten (Balanced Datenset). Somit kann zum Beispiel eine Metrik wie 'Accuracy' verwendet werden, um die Klassifikation der Modelle zu beurteilen und zu vergleichen. Die 10'000 Bilder in den Testdaten sind ebenfalls gleich verteilt.
    
</div>

Mittelwerte und Standardabweichungen des Trainingdatensets (RGB):
Benötigt um die Daten zu normalisieren

In [ ]:
# Erstelle Datenset mit Tensoren für Berechnunen
transform = transforms.Compose([transforms.ToTensor()])
train_data = torchvision.datasets.CIFAR10(data_path, train=True, download=False, transform=transform)
test_data = torchvision.datasets.CIFAR10(data_path, train=False, download=False, transform=transform)

In [ ]:
def calc_mean_std_dataset(train_data, print_info=True):
    '''
    Berechnet den Mittelwert und Standardabweichung für alle Bilder
    '''
    num_images = len(train_data)

    pixel_values = np.zeros((num_images, 3, 32, 32), dtype=np.float32)

    for i in range(num_images):
        image, _ = train_data[i]
        pixel_values[i] = image.numpy()

    mean = np.mean(pixel_values, axis=(0, 2, 3))
    std = np.std(pixel_values, axis=(0, 2, 3))

    if print_info:
        print("RGB-Mittelwerte:", mean)
        print("RGB-Standardabweichungen:", std)

calc_mean_std_dataset(train_data, True)

### 2. Preprocessing der Daten  
Folgend werden die Daten in einem Preprocessing Schritt für die Modelle vorbereitet:

In [ ]:
def get_cross_validation_loaders(dataset, kfold, train_batch_size, test_batch_size):
    '''
    todo: fix CV Fehler
    '''
    cv_train_loader = []
    cv_test_loader = []

    for fold, (train_idx, test_idx) in enumerate(kfold.split(dataset)):
        # train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
        # val_subsampler = torch.utils.data.SubsetRandomSampler(test_idx)       
        
        # train_loader = torch.utils.data.DataLoader(dataset, 
        #                 batch_size=train_batch_size, 
        #                 sampler=train_subsampler,
        #                 shuffle=True)
        # val_loader = torch.utils.data.DataLoader(dataset,
        #                 batch_size=test_batch_size,
        #                 sampler=val_subsampler,
        #                 shuffle=False)
        

        train_subsampler = torch.utils.data.Subset(dataset, train_idx)
        val_subsampler = torch.utils.data.Subset(dataset, test_idx)

        train_loader = torch.utils.data.DataLoader(train_subsampler, 
                        batch_size=train_batch_size, 
                        shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_subsampler,
                        batch_size=test_batch_size,
                        shuffle=False)
        
        cv_train_loader.append(train_loader)
        cv_test_loader.append(val_loader)
    return cv_train_loader, cv_test_loader

# function preprocessing
def preprocessing_cifar10(path='./data/', train_batch_size=32, test_batch_size=32, normalize='zero_one',
                          norm_mean=(0.4914009  , 0.548215896, 0.4465308), 
                          norm_std=(0.24703279 , 0.24348423 , 0.26158753), 
                          download=True, print_info=False, cv=False, k_folds=5, 
                          set_seed=42):
    
    if print_info: print(f'------------'), print(f'Preprocessing start')

    if normalize == 'zero_one':
        # transform tensor to normalized range [0, 1], 0=schwarz
        transform = transforms.Compose([transforms.ToTensor()])
    elif normalize == 'minusone_one':
        # transform tensor to normalized range [-1, 1], 0=schwarz
        transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])
    elif normalize == 'None':
        transform = transforms.Compose([])
    else:
        raise ValueError('Nomalize must be either "zero_one", "minusone_one" or "None"') 
    
    if cv:
        dataset_train = torchvision.datasets.CIFAR10(root=path, train=True, download=download, transform=transform)        
        dataset_test = torchvision.datasets.CIFAR10(root=path, train=False, download=download, transform=transform)
        dataset = dataset = torch.utils.data.ConcatDataset([dataset_train, dataset_test])

        if print_info: 
            print(f'Data transformed: {normalize}')
            calc_mean_std_dataset(dataset, print_info)
        
        kfold = KFold(n_splits=k_folds, shuffle=True, random_state=set_seed)        

        cv_train_loader, cv_test_loader = get_cross_validation_loaders(
            dataset, kfold, train_batch_size, test_batch_size)
        
        if print_info: print(f'Dataloader created with {train_batch_size=}, {test_batch_size=}')
        if print_info: print(f'Preprocessing done'), print('------------')

        return cv_train_loader, cv_test_loader

    else:
        # CIFAR10: 50000 32x32 color images in 10 classes, with 5000 images per class
        train_dataset = torchvision.datasets.CIFAR10(root=path, train=True,
                                                download=download, transform=transform)
        test_dataset = torchvision.datasets.CIFAR10(root=path, train=False,
                                            download=download, transform=transform)
        if print_info: 
            print(f'Data transformed: {normalize}')
            calc_mean_std_dataset(train_dataset, print_info)

        # dataloader
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

        if print_info: print(f'Dataloader created with {train_batch_size=}, {test_batch_size=}')
        if print_info: print(f'Preprocessing done'), print('------------')   

        return train_dataset, test_dataset, train_loader, test_loader


In [ ]:
data_path = './data/'
batch_size = 32
transform = 'zero_one' # minusone_one

train_dataset, test_dataset, train_loader, test_loader = preprocessing_cifar10(
    path=data_path, 
    train_batch_size=batch_size,
    test_batch_size=batch_size,
    normalize=transform,  
    download=False,  
    print_info=True
    )

In [ ]:
# Hyper-parameters 
data_path = './data/'
batch_size = 32
transform = 'zero_one' # minusone_one

cv_train_loader, cv_test_loader = preprocessing_cifar10(
    train_batch_size=batch_size,
    test_batch_size=batch_size,
    normalize=transform,
    download=False,  
    print_info=False, cv=True
    )

print(f'K Fold CV: {len(cv_train_loader)}')

<div class="alert alert-block alert-success">

Aus den Berechnungen des Mittelwert und Standardabweichung lässt sich schliessen dass die Daten von Cifar-10 bereits normalisiert wurden auf Werte zwischen [0,1]. Mit der Transformation im Preprocessing `transforms.Normalize(mean, std)` können die Werte in den Bereich [-1, 1] gesetzt werden. Welcher Wertebereich die bessere Wahl ist, ist Situationsbedingt (z.B. verwendete Aktivierungsfunktion). Eine Zentrierung um Null kann für Netze Vorteile haben, hingegen sind Werte zwischen [0,1] besser zu interpretieren (0=schwarz, 1=RGB 255). Vorerst soll mit dem Wertebereich [0,1] gearbeitet werden.

</div>

# Schritt 3: Aufbau Modellierung  

<div class="alert alert-block alert-info">

1. Lege fest, wie (mit welchen Metriken) Du die Modelle evaluieren möchtest. Berücksichtige auch den Fehler in der Schätzung dieser Metriken.
    
</div>

<div class="alert alert-block alert-success">

Die Klassengrössen sind ausbalanciert, daher ist für die Metrik **Accuracy** für die Modell Beurteilung geeignet und soll hier für die Modellbewertung zum Einsatz kommen:

$$ Accuracy = \frac{Anzahl\ korrekte\ Klassifizierungen}{Total\ Klassifizierungen}  $$

Für eine Klassifizierung von mehr als zwei Klassen eignet sich **CrossEntropyLoss**. Mehr dazu [hier](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html):

$$ \text{CrossEntropyLoss} = -\sum_{i=1}^{N} \left[ y_i \log(\hat{y_i}) + (1 - y_i) \log(1 - \hat{y_i}) \right] $$


Die Accuracy berechnet eine Schätzung. Bei der Initialisierung der Modellgewichte werden Zufallswerte verwendet. Auch die Wahl der Bilder innerhalb der Batchsize wird durch `shuffle=True` zufällig getroffen *(siehe `Preprocessing`)*. Somit varriert die Accuracy nach jedem Modeltraining ein wenig. Cross-Validation würde sich hier anbieten, um auf k-folds unterschiedliche Modelle zu erstellen. Auch ein Modell mehrmals ausführen, zu unterschiedlichen Seeds, wäre denkbar. Mit dem berechneten Mittelwert $\mu$ und der Standardabweichung $\sigma$ kann ein Fehlerabschätzung gemacht werde.  
$$ Fehler_{range} = [\mu - \sigma; \mu + \sigma]$$

</div>

<div class="alert alert-block alert-info">

2. Implementiere Basisfunktionalität, um Modelle zu trainieren und gegeneinander zu evaluieren. Wie sollen die Gewichte initialisiert werden?
    
</div>

<div class="alert alert-block alert-success">

Die Methode wie die Intitialisierung der Gewichte stattfindet hat Einfluss wie schnell das Modell konvergiert und hilft die Probleme wie `vanishing` oder `exploding` Gradienten abzuschwächen. Kleine zufällige Werte führen zu einem effizienteren Training. Grosse Werte führen zu Problemenn bei dem das Modell nicht oder nur sehr langsam konvergiert. Je nach Problemstellung und Modelarchitekture können unterschiedliche Initialisierungsmethoden verwendet werden

Mit der Verwendung von `nn.init` (Pytorch) stehen zum Beispiel folgende Optionen zur Verfügung:
1. **Uniform initialization** (help prevents: vanishing gradient, can suffer: exploding gradient)
1. **Xavier initialization** (help prevent: vanishing gradient)
1. **Kaiming initialization** (help prevent: vanishing gradient, account activation function)
1. **Normal initialization** (help prevent: exploding gradient)
1. **Zeros initialization** (can suffer: slow converge, vanishing gradient)
1. **One’s initialization** (can suffer: slow converge, vanishing gradient)

Die genaue Beschreibung der Intitialisierungen können [hier](https://www.geeksforgeeks.org/initialize-weights-in-pytorch/) gefunden werden. Eine eigene Custom-Option stellt Pytorch auch zur Verfügung.

</div>

### Definition Helper Functions  

In [ ]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
set_seed()

def plot_loss_epoch(num_epochs, loss, figsize=(8, 4)):
    figure = plt.figure(figsize=figsize)
    plt.plot(np.arange(num_epochs), loss)
    plt.xticks(np.arange(num_epochs))
    plt.title('Loss Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()

def calc_true_predictions(output_model, true_labels):
    _, predicted = torch.max(output_model.data, 1)
    return (predicted == true_labels).sum().item()

def measure_model_time(start_time, calc='min'):
    model_time = np.round(((time.time() - start_time) / 60), 2)  # from seconds to minute
    return model_time

def play_sound(typ=0):
    # play 'finish' sound
    if typ==0:
        winsound.PlaySound('../01_Dokumentation/win_sounds/beep.wav', winsound.SND_ASYNC)
    if typ==1:
        winsound.PlaySound('../01_Dokumentation/win_sounds/beep2.wav', winsound.SND_ASYNC)

def plot_init_weights(model, method, figsize=(6, 5)):
    l1_weights = model.state_dict()['linear1.weight'].numpy()
    l2_weights = model.state_dict()['linear2.weight'].numpy()
    fig, ax = plt.subplots(1,2, figsize=figsize)

    ax[0].hist(l1_weights.flatten(), bins=50)
    ax[1].hist(l2_weights.flatten(), bins=50)

    ax[0].set_title("Layer 1", fontsize=8)
    ax[0].set_xlabel("Gewichtswert", fontsize=6)
    ax[0].set_ylabel("Anzahl", fontsize=8)
    ax[0].tick_params(axis='y', labelsize=6)
    ax[0].tick_params(axis='x', labelsize=6)
    ax[1].set_title("Layer 2", fontsize=8)
    ax[1].set_xlabel("Gewichtswert", fontsize=6)
    ax[1].tick_params(axis='y', labelsize=6)
    ax[1].tick_params(axis='x', labelsize=6)
    plt.suptitle(f'Verteilung Initialisierungs Methode: {method}', fontsize=8)
    plt.show()

def test_eval_plot():
    num_epochs = 10
    n_loss_epochs = (0.05 * np.sqrt(np.arange(10))) * -1
    n_correct_train = 0.1 * n_loss_epochs**4
    n_correct_test = 0.05 * n_loss_epochs**4
    # create dataloader
    train_dataset, test_dataset, train_loader, test_loader = preprocessing_cifar10(batch_size=4,
                                                                                norm_mean=(0.5, 0.5, 0.5), 
                                                                                norm_std=(0.5, 0.5, 0.5),
                                                                                download=False,  
                                                                                print_info=False)
    eval_model(num_epochs, n_loss_epochs, n_correct_train, n_correct_test,
                train_loader, test_loader)
# test_eval_plot()   

### Definitionen Evaluation  
Die Ergebnisse eines Modeltrainings werden in einem Plot festgehalten und gespeichert. Für die Diskusion der Modelle werden jedoch die Plots von wandb.ai verwendet.

In [ ]:
def eval_model(num_epochs, n_loss_epochs, n_val_loss_epochs, 
                n_acc_train_epochs, n_acc_test_epochs, train_loader, test_loader, 
                n_loss_batches=None, group_name='group_name', tag_name='tag_name', figsize=(8,5),
                print_info=True, save_image=False):
    
    epoch = np.arange(num_epochs)

    fig, ax1 = plt.subplots(figsize=figsize)
    ax2 = ax1.twinx()

    ax1.plot(epoch, n_loss_epochs, color= 'grey', linestyle='--', label='Train Loss')
    ax1.plot(epoch, n_val_loss_epochs, color= 'grey', label='Test Loss')
    ax2.plot(epoch, n_acc_train_epochs, color='steelblue', label='Accuracy Train')
    ax2.plot(epoch, n_acc_test_epochs, color='coral', label='Accuracy Test')
    
    ax1.set_xticks(epoch)
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss') 
    ax2.set_ylabel('Accuracy')
    ax1.tick_params(axis='x', labelsize=8)
    ax1.tick_params(axis='y', labelsize=8)
    ax2.tick_params(axis='y', labelsize=8)
    ax1.legend(loc='upper center', bbox_to_anchor=(0.25, -0.12), ncol=3)    
    ax2.legend(loc='upper center', bbox_to_anchor=(0.72, -0.12), ncol=3)
    
    plt.suptitle(f'Training: {group_name} ')
    cur_date = datetime.now().strftime("%d.%m.%Y_%H%M")
    plt.title(f'{cur_date} - Tags: {tag_name}', size=6)
    plt.grid()
    if save_image: 
            plt.savefig(f'./plots/eval_model_plots/{cur_date}_{group_name}_{tag_name}.png')
    plt.show()

    if print_info:
        print(f'Accuracy Train {n_acc_train_epochs[-1]:2f}')
        print(f'Accuracy Test {n_acc_test_epochs[-1]:4f}')
        print(f'Loss Train {n_loss_epochs[-1]:2f}')
        print(f'Loss Test {n_val_loss_epochs[-1]:2f}')

    return

### Definitionen Training 

In [ ]:
from datetime import datetime

# Hyperparameters example
config = {
    "name": "MLP", 
    "epochs": 20,   
    "train_batch_size": 32, 
    "test_batch_size": 32,
    "dataset": "CIFAR-10",
    "lr": 1e-3, 
    "optimizer": 'SGD',
    "Regularisierung": 'None',  # 'None', 'L1', 'L2',
    "L1_lambda":0,
    "L2_weight_decay":0,
    "loss_func": 'CrossEntropyLoss',
    "activation": "ReLU",
    "image_size": 32,
    "cross_validation": False,
    "is_test_batch": True,
    "start_time": datetime.now().strftime("%d.%m.%Y_%H%M"),
    "num_workers": 0,
    "normalize":"zero_one",
    "init_w_method":"kaiming_norm",  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
    "hidden_layer_sizes": [128],
    "filter_sizes": [32, 64],
    "dense_layers": [512],
    "kernel_sizes": 3,
    "padding": 1,
    "stride": 1,
    "pool_sizes": [],
    "dropout": 0,
    "norm_mean": (0.5, 0.5, 0.5),
    "norm_std": (0.5, 0.5, 0.5),
    "num_classes": 10,
    "save_eval_image": True,
    "set_seed": 42
}


# Train model function
def train_model(
        model, train_loader, test_loader, config, tags=['tag_name'], group='group_name',
        print_info=False, plot_eval=True, sound=False, write_wandb=True, fold=0):      
    
    set_seed(config['set_seed'])

    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
    if print_info: print(f'------------'), print(f'Starten des Trainings auf Device {device}')

    # Model to device
    model = model.to(device)
    model.train()

    # define Loss and Optimizer
    if config['loss_func'] == 'CrossEntropyLoss':
        criterion = nn.CrossEntropyLoss()
        if print_info: print(f"Loss Funktion: {config['loss_func']}")

    if config['optimizer'] == 'SGD':        
        optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'])
        if print_info: print(f"Optimizer: {config['optimizer']} mit lr: {config['lr']}")
    if config['optimizer'] == 'SGD' and config['Regularisierung'] == 'L2':        
        optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'], weight_decay=config['L2_weight_decay'] )
        if print_info: print(f"Optimizer: {config['optimizer']} mit lr: {config['lr']}, w_decay:{config['L2_weight_decay']}")
        
    if config['optimizer']  == 'Adam':        
        optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
        if print_info: print(f"Optimizer: {config['optimizer']} mit lr: {config['lr']}")
    if config['optimizer']  == 'Adam' and config['Regularisierung'] == 'L2':        
        optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['L2_weight_decay'])
        if print_info: print(f"Optimizer: {config['optimizer']} mit lr: {config['lr']}")

    # Initialize wandb
    if write_wandb: 
        cv_model_name = f"{config['name']}-CV-{fold+1}/{config['n_folds']}-{config['epochs']}-epochs-{config['optimizer']}-{config['start_time']}"
        model_name = f"{config['name']}-{config['epochs']}-epochs-{config['optimizer']}-{config['start_time']}"
        wandb.init(
            project="del-mc1",
            entity='manuel-schwarz',
            group=group,
            name=cv_model_name if config['cross_validation'] else model_name,
            tags=tags + (['is_test_batch'] if config['is_test_batch'] else []),
            config=config
        )

    # --------------------- train loop --------------------------
    if write_wandb:
        wandb.watch(model)

    n_loss_epochs = []
    n_val_loss_epochs = []
    n_loss_all_batches = []
    n_val_loss_all_batches = []
    n_correct_train_epochs = []
    n_correct_test_epochs = []
    n_acc_train_epochs = []
    n_acc_test_epochs = []

    best_train_loss = 0
    best_val_loss = 0
    best_train_acc = 0
    best_test_acc = 0
    best_epoch = 0

    loop = range(config["epochs"])
    epoch_loop = tqdm(loop, desc="Epochs", position=0, leave=True)

    for epoch in epoch_loop:
        true_train = 0
        n_loss_batch = []

        for i, (images, labels) in enumerate(train_loader):
            if config['is_test_batch'] and i > 1:
                break
            # origin shape: [4, 3, 32, 32] = 4, 3, 1024
            # input_layer: 3 input channels, 6 output channels, 5 kernel size
            images, labels = images.to(device), labels.to(device)

            # Backward and optimize
            optimizer.zero_grad()
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            # L1-Regularisierung
            if config["Regularisierung"] == 'L1':
                l1_loss = sum(p.abs().sum() for p in model.parameters())
                loss = loss + config["L1_lambda"] * l1_loss
                
            loss.backward()
            optimizer.step()

            n_loss_batch.append(loss.item())
            true_train += calc_true_predictions(outputs, labels)
            # acc_train = true_train / len(train_loader.dataset)
            n_loss_all_batches.append(loss.item())

            inner_progress = f"{i+1}/{len(train_loader)}"
            epoch_loop.set_description(f"Epochs (Batch: {inner_progress})")
            epoch_loop.refresh()

        # save results
        acc_train = true_train / len(train_loader.dataset)
        epoch_loss = np.mean(n_loss_batch)

        n_correct_train_epochs.append(true_train)        
        n_acc_train_epochs.append(acc_train)  
        n_loss_epochs.append(epoch_loss)

        # ---------- eval
        model.eval()
        true_test_val = 0 
        n_val_loss_batch = []
        # n_val_pred = []

        with torch.no_grad():
            for i, (images, labels) in enumerate(test_loader):
                if config['is_test_batch'] and i > 1:
                    break
                images, labels = images.to(device), labels.to(device)
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                n_val_loss_batch.append(loss.item())
                true_test_val += calc_true_predictions(outputs, labels)
                n_val_loss_all_batches.append(loss.item())              
                # _, predicted = torch.max(outputs, 1)  # prediction speichern todo

        # save results
        acc_test = true_test_val / len(test_loader.dataset)
        epoch_val_loss = np.mean(n_val_loss_batch)

        n_correct_test_epochs.append(true_test_val)
        n_acc_test_epochs.append(acc_test) 
        n_val_loss_epochs.append(epoch_val_loss)

        model.train()

        # best scores
        if epoch_loss < best_train_loss:
            best_train_loss = epoch_loss

        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss

        if acc_train > best_train_acc:
            best_train_acc = acc_train

        if acc_test > best_test_acc:
            best_test_acc = acc_test
            best_epoch = epoch

        # log metrics to wandb
        if write_wandb:
            wandb.log({
                "loss epoch": epoch_loss, 
                "validation loss epoch": epoch_val_loss, 
                "acc_train": acc_train, 
                "acc_test": acc_test,
                })
    if write_wandb:
            wandb.log({
                "best_epoch": best_epoch,
                "best_train_loss": best_train_loss,
                "best_val_loss": best_val_loss,
                "best_acc_train": best_train_acc,
                "best_acc_test": best_test_acc,
                })

    acc_train = n_correct_train_epochs[-1] / len(train_loader.dataset)
    acc_test = n_correct_test_epochs[-1] / len(test_loader.dataset)

    if plot_eval:
        eval_model(config['epochs'], n_loss_epochs, n_val_loss_epochs, 
                    n_acc_train_epochs, n_acc_test_epochs, train_loader, test_loader,
                    group_name=group, tag_name=tags, print_info=print_info, 
                    save_image=config['save_eval_image']
                    )
        
    print(
            f"Epoch {epoch + 1}/{config['epochs']}, \
            Loss: {round(epoch_loss, 5)}, Validation Loss: {round(epoch_val_loss, 5)} \
            Acc: {round(acc_train, 5)}, Validation Acc: {round(acc_test, 5)}"
            )
    
    if sound: play_sound(1)

    if write_wandb: 
        wandb.finish()
        time.sleep(5)  # wait for wandb.finish

    print('Finished Training')
    return epoch_loss, epoch_val_loss, acc_train, acc_test


def model_trainer(model, config=config, group='group_name', tags=['tag1'],
                print_info=False, plot_eval=True, sound=True, 
                write_wandb=True, download=False):
    
    if config['cross_validation']:
        cv_train_loader, cv_test_loader = preprocessing_cifar10(
            train_batch_size=config["train_batch_size"],
            test_batch_size=config["test_batch_size"],
            normalize=config['normalize'],
            download=download,  
            print_info=print_info, cv=True,
            set_seed=config['set_seed']
            )
        
        cv_train_acc = []
        cv_test_acc = []

        for fold, (train_loader, test_loader) in enumerate(zip(cv_train_loader, cv_test_loader)):
            _, _, train_acc, test_acc = train_model(model, train_loader, test_loader, config=config,
                    group=group,
                    tags=tags,
                    print_info=print_info, plot_eval=plot_eval, sound=sound, 
                    write_wandb=write_wandb, fold=fold
                    )
            
            cv_train_acc.append(train_acc)
            cv_test_acc.append(test_acc)

        cv_train_acc_mean = np.mean(cv_train_acc)
        cv_test_acc_mean = np.mean(cv_test_acc)

        return cv_train_acc_mean, cv_test_acc_mean, cv_train_acc, cv_test_acc

    else:
        # create dataloader
        _, _, train_loader, test_loader = preprocessing_cifar10(
            train_batch_size=config["train_batch_size"],
            test_batch_size=config["test_batch_size"],
            normalize=config['normalize'],
            download=download,  
            print_info=print_info,
            set_seed=config['set_seed']
            )

            # train model
        _, _, train_acc, test_acc = train_model(model, train_loader, test_loader, config=config,
                    group=group,
                    tags=tags,
                    print_info=print_info, plot_eval=plot_eval, sound=sound, 
                    write_wandb=write_wandb
                    )
        return train_acc, test_acc, 0, 0

### Einfacher Modell Test  
Folgend soll ein einfaches Modell mit einem hidden Layer erstellt werden um die Basisfunktionen von `model_trainer()` zu testen. 


In [ ]:
# simple model Class
class MLPNet_hl1(nn.Module):
    def __init__(self, hidden_l_1:list, act_fn=F.relu, dropout=0, init_methode:str='kaiming_norm'):
        super(MLPNet_hl1, self).__init__()
        self.linear1 = nn.Linear(3*32*32, hidden_l_1[0])  # input.shape = (n, 3, 32, 32)
        self.linear2 = nn.Linear(hidden_l_1[0], 10)
        self.dropout = nn.Dropout(dropout)
        self.activation = act_fn 

        for linear_weight in [self.linear1.weight, self.linear2.weight]:          
            if init_methode == 'uniform':
                torch.nn.init.uniform_(linear_weight)
            if init_methode == 'xavier':
                torch.nn.init.xavier_uniform_(linear_weight)
            if init_methode == 'normal':
                torch.nn.init.normal_(linear_weight, mean=0, std=1)
            if init_methode == 'kaiming_norm':
                torch.nn.init.kaiming_normal_(linear_weight, nonlinearity='relu')

    def forward(self, x): # x.shape = (n, 3, 32, 32)
        x = x.view(-1, 3*32*32) # x.shape = (n, 3072)
        x = self.activation(self.linear1(x)) # x.shape = (3072, 128)
        x = self.dropout(x)
        x = self.linear2(x) # x.shape = (128, 10)
        return x

Initilisierung Gewichte: 
Da unterschiedliche Problemstellung verschiedene Initialisierungen erfordern, sollen mehrere Methoden von Pytorch ausprobiert werden um die Gewichte zu initialisiern: Folgende Grafik zeigt die Gewichte des ersten (initial) und zweiten Layers:
1. `uniform`: torch.nn.init.uniform_(linear_layer.weight)
1. `xavier`: torch.nn.init.xavier_uniform_(linear_layer.weight)
1. `normal`: torch.nn.init.normal_(linear_layer.weight, mean=0, std=1)
1. `kaiming_norm`: torch.nn.init.kaiming_normal_(linear_layer.weight, nonlinearity="relu")

In [ ]:
method_lst = ['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
# create model
for method in method_lst:
    model = MLPNet_hl1(hidden_l_1=[64], init_methode=method)
    plot_init_weights(model, method, figsize=(6, 3))

Die Verteilungen der Gewichte mit den Methoden `uniform` und  `xavier` sind sehr ähnlich. Mit Xavier ist die X-Skala unterschiedlich, da die Gewichte so skalliert werden dass die Varianz des Output der Varianz des Inputs entspricht. Bei `normal` und `kaiming_norm` besteht das gleiche Prinzip, die Skallierung der Gewichte, weiter wird die Aktivierungsfunktion berücksichtigt.

In [ ]:
if False:
    from datetime import datetime
    # Hyperparameters  
    config = {
        "name": "MLP-test1", 
        "epochs": 1,   
        "train_batch_size": 32, 
        "test_batch_size": 32,
        "dataset": "CIFAR-10",
        "lr": 1e-3, 
        "optimizer": 'SGD',
        "Regularisierung": 'None',  # 'None', 'L1', 'L2',
        "L1_lambda":0,
        "L2_weight_decay":0,
        "loss_func": 'CrossEntropyLoss',
        "loss_func": 'CrossEntropyLoss',
        "activation": "ReLU",
        "image_size": 32,
        "cross_validation": False,
        "n_folds": 5,
        "is_test_batch": True,
        "start_time": datetime.now().strftime("%d.%m.%Y_%H%M"),
        "num_workers": 0,
        "normalize":"zero_one",
        "init_w_method":"kaiming_norm",  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
        "hidden_layer_sizes": [128],
        "filter_sizes": [32, 64],
        "dense_layers": [512],
        "kernel_sizes": 3,
        "padding": 1,
        "stride": 1,
        "pool_sizes": [],
        "dropout": 0,
        "norm_mean": (0.5, 0.5, 0.5),
        "norm_std": (0.5, 0.5, 0.5),
        "num_classes": 10,
        "save_eval_image": True,
        "set_seed": 42
    }

    # create model
    model2 = MLPNet_hl1(
        hidden_l_1=config["hidden_layer_sizes"], 
        act_fn=F.relu,
        dropout=0,
        init_methode=config["init_w_method"],
        )

    cv_train_acc_mean, cv_test_acc_mean, _, _ = model_trainer(model2, config=config,
                    group='MLP_CV_test2',
                    tags=['MLP', 'cv-test'],
                    print_info=False, plot_eval=False, sound=True, 
                    write_wandb=False
                    )


# Schritt 4: Evaluation  

<div class="alert alert-block alert-warning">

Bei der Evaluation ist darauf zu achten, dass das Vorgehen stets möglichst reflektiert erfolgt und versucht wird, die Ergebnisse zu interpretieren. Am Schluss soll auch ein Fazit gezogen werden, darüber welche Variante am besten funktioniert.  

</div>

<div class="alert alert-block alert-info">

**1. Training mit SGD, ohne REG, ohne BN:**  
Untersuche verschiedene Modelle unterschiedlicher Komplexität, welche geeignet sein könnten, um das Klassifikationsproblem zu lösen. Verwende Stochastic Gradient Descent - ohne Beschleunigung, ohne Regularisierung (REG) und ohne Batchnorm (BN).  

a. Für jedes Modell mit gegebener Anzahl Layer und Units pro Layer führe ein sorgfältiges Hyper-Parameter-Tuning durch (Lernrate, Batch-Grösse). Achte stets darauf, dass das Training stabil läuft. Merke Dir bei jedem Training, den Loss, die Performance Metrik(en) inkl. Schätzfehler, die verwendete Anzahl Epochen, Lernrate und Batch-Grösse.

b. Variiere die Anzahl Layer und Anzahl Units pro Layer, um eine möglichst gute Performance zu erreichen. Falls auch CNNs (ohne Transfer-Learning) verwendet werden variiere auch Anzahl Filter, Kernel-Grösse, Stride, Padding.

c. Fasse die Ergebnisse zusammen in einem geeigneten Plot, bilde eine Synthese und folgere, welche Modell-Komplexität Dir am sinnvollsten erscheint.  

</div>


## **MLP** Hyperparameter Suche

Für ein gutes Modell müssen optimale Hyperparamter gefunden werden. Folgende werden MLP-Modelle mit unterschiedlichen Anzahl und Grösse von Layers erstellt. Zu jedem Modell soll weiter den Einfluss von Lernrate und Batchsize untersucht werden:

**Ablauf pro Modell:**  
1. Der Umfang sowie die Anzahl der hidden Layers werden gesetzt
1. Pro Modell werden verschiedene Lernraten untersucht
1. Pro Modell werden verschiedene Grössen von Layern untersucht
1. Pro Modell werden verschiedene Batchgrössen untersucht

Das Tracking von Loss und Metriken werden mit wandb für jedes Training aufgezeichnet. Bildauschnitte des Trainings wurden zur Diskusion hinzugefügt.

---

**MLP mit einem hidden Layer**  
In den folgenden Experimenten soll die Lernrate, die Grösse des hidden Layers und die Batchsize untersucht werden:

|Experiment| Lernrate | Grösse hL1 | Batchsize |
|---|----------|----------|----------|
|1| 1e-1, 1e-2, 1e-3, 1e-4, 1e-5 | 128*  | 32*  |
|2| 1e-2** | 64, 128, 256, 512, 1024 |  32* |
|3| 1e-2** | 1024**  | 4, 16, 32, 64, 128  |

Info:  
(\*) gewählter Startwert    
(\*\*) optimaler Wert    


In [ ]:
class MLPNet_hl1(nn.Module):
    def __init__(self, h_layer_sizes:list, input_size=3*32*32, act_fn=F.relu, dropout=0, 
                init_methode:str='kaiming_norm', output_size=10):
        super(MLPNet_hl1, self).__init__()
        self.linear1 = nn.Linear(input_size, h_layer_sizes[0])  # input.shape = (n, 3, 32, 32)
        self.linear2 = nn.Linear(h_layer_sizes[0], 10)
        self.dropout = nn.Dropout(dropout)
        self.activation = act_fn 

        for linear_weight in [self.linear1.weight, self.linear2.weight]:          
            if init_methode == 'uniform':
                torch.nn.init.uniform_(linear_weight)
            if init_methode == 'xavier':
                torch.nn.init.xavier_uniform_(linear_weight)
            if init_methode == 'normal':
                torch.nn.init.normal_(linear_weight, mean=0, std=1)
            if init_methode == 'kaiming_norm':
                torch.nn.init.kaiming_normal_(linear_weight, nonlinearity='relu')

    def forward(self, x): # x.shape = (n, 3, 32, 32)
        x = x.view(-1, 3*32*32) # x.shape = (n, 3072)
        x = self.activation(self.linear1(x)) # x.shape = (3072, 128)
        x = self.dropout(x)
        x = self.linear2(x) # x.shape = (128, 10)
        return x

In [ ]:
if False:
    from datetime import datetime

    #for lr in [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]:
    #for hl_size in [64, 128, 256, 512, 1024]:
    #for bt_size in [4, 16, 32, 64, 128]:
    for seed in [12, 16, 42, 64, 71]:
        # Hyperparameters  
        config = {
            "name": "MLP-hL1", 
            "epochs": 15,   
            "train_batch_size": 32, 
            "test_batch_size": 32,
            "dataset": "CIFAR-10",
            "lr": 1e-2, # default 1e-3
            "optimizer": 'SGD',
            "Regularisierung": 'None',  # 'None', 'L1', 'L2',
            "L1_lambda":0,
            "L2_weight_decay":0,
            "loss_func": 'CrossEntropyLoss',
            "loss_func": 'CrossEntropyLoss',
            "activation": "ReLU",
            "image_size": 32,
            "cross_validation": False,
            "n_folds": 5,
            "is_test_batch": False,
            "start_time": datetime.now().strftime("%d.%m.%Y_%H%M"),
            "num_workers": 0,
            "normalize":"zero_one",
            "init_w_method":"kaiming_norm",  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
            "hidden_layer_sizes": [1024],
            "filter_sizes": [32, 64],
            "dense_layers": [512],
            "kernel_sizes": 3,
            "padding": 1,
            "stride": 1,
            "pool_sizes": [],
            "dropout": 0,
            "norm_mean": (0.5, 0.5, 0.5),
            "norm_std": (0.5, 0.5, 0.5),
            "num_classes": 10,
            "save_eval_image": True,
            "set_seed": seed  # [12, 16, 42, 64, 71]
        }

        # create model
        mlp_h1 = MLPNet_hl1(
            h_layer_sizes=config["hidden_layer_sizes"], 
            act_fn=F.relu,
            dropout=0,
            init_methode=config["init_w_method"],
            )

        cv_train_acc_mean, cv_test_acc_mean, _, _ = model_trainer(mlp_h1, config=config,
                        group=f'MLP_hL1_seed_val',  # _5f_cv, _seed_val
                        tags=['MLP', 'seed val'],  # 5fold cv, seed val
                        print_info=False, plot_eval=True, sound=True, 
                        write_wandb=True
                        )


**Unterschiedliche Lernraten**

<img src="../01_Dokumentation/wandb_images/MLP_hl1_lr.PNG" width="800" height="400">

**Unterschiedliche Layergrösse**

<img src="../01_Dokumentation/wandb_images/MLP_hl1_layer_size.PNG" width="900" height="400">

**Unterschiedliche Batchgrösse**

<img src="../01_Dokumentation/wandb_images/MLP_hl1_batch_size.PNG" width="900" height="400">

<div class="alert alert-block alert-success">

**Einfluss der Learningrate:**  
Die Lernrate bestimmt, wie stark die Modellgewichte in jedem Trainingsschritt angepasst werden. $$ W_{neu}= W_{alt} - Lernrate \cdot Gradient $$
Bei einer zu hohen Lernrate kann es passieren, dass das optimale globale Minimum übersprungen wird. Das Training könnte auch divergieren statt konvergieren. Bei einer zu tiefen Lernrate kann das Training an einem lokalen Minimum hängen bleiben und das globale Minimum nicht erreichen. Auch findet der Trainingsprozess weniger schnell statt. Das ist auch der Grund der tieferen Accuracy bei den verschiedenen Lernraten, tiefere Lernraten müssten mit mehr Epochen trainiert werden. Bei $lr=1^{-2}$ erreicht das Training die höchste Accuracy, wobei der Verlauf etwas weniger stabil aussieht. Die Lernraten $1^{-1}$ und $1^{-3}$ sind sehr ähnlich. Ein Optimum wurde bei $1^{-2}$ gefunden. 


**Einfluss Grösse des Hidden Layers:**  
Ein Modell mit einem hidden Layer verbindet den Input Layer mit dem Output Layer. In unserem Fall $Input\ Layer (Bild): 3*32*32 = 3072 -> hidden\ Layer -> Output\ Layer (Anzahl\ Klassen) = 10$. Die Lernrate wurde auf $1^{-2}$ gesetzt. In der  Entwicklung von Trainings Loss und der Training Accuracy zeigt sich ein sehr ähnliches Verhalten. Die Test Accuracy zeigt, dass eine sehr tiefe Layergrösse weniger gut geeignet ist. Die Unterschiede ab Grösse 256 sind weniger markant. Eine Layergrösse von 1024 zeigte die beste Accuracy. Grössere Layer können besser komplexe Funktionen abbilden und dadurch mehr Muster aus den Daten erkennen. Es bedeutet aber auch mehr Gewichte und eine höherer Trainingszeit. Die Layergrösse hat auch Einfluss auf den Bias-Varianz Tradeoff. 

**Einfluss der Batchgrösse:**  
Mit einer kleineren Batchgrösse werden die Gewichte des Modells häufiger angepasst. Das kann das Training schneller aber auch instabiler machen. Eine grössere Batchsize kann eine genauere Schätzung des Gradienten erstellen. Ähnlich wie bei einer tiefen Lernrate, könnten aber lokale Minimas weniger gut übersprungen werden. Die Wahl, spezifisch bei Bilddaten, wird auch stark durch den vorhandenen GPU-Speicher begrenzt. In unseren Versuchen sehen wir ähnliche Ergebnisse in der Test Accuracy wobei die Batchgrössen 16 und 32 die höchste Accuracy erreichten. Interessant war dass das Modell mit der Batchgrösse 4 für 15 Epochen 24 Minuten benötigte, gegenüber von Batchgrösse 16 = 8 Minuten, Batchgrösse 32 = 5 Minuten, Batchgrösse 64 = 4 Minuten und Batchgrösse 128 = 3 Minuten. Die häufige Anpassungen der Gewichte haben deutlichen Einfluss auf die Trainingszeit. Für das Modell soll mit der Batchgrösse 32 fortgefahren werden.

</div>

**MLP_hl1 Modell Evaluation**:  
Für die optimierten Parameter soll das Modell mit 5Fold-Cross-Validation und mit unterschiedlichen Seeds auf Stabilität geprüft werden:

*Bemerkung: Die Cross-Validation Option hat einen Fehler der noch nicht behoben ist. CV1 funktioniert, aber für die CV2-5 passen die Train- Testloader nicht überrein und führen zu inkorrekten Losses und Metriken. Dadurch wird die Performance stark überschätzt. Für den Schätzfehler sollen die Modelle zu den unterschiedlichen Seeds betrachtet werden.* 

<img src="../01_Dokumentation/wandb_images/MLP_hl1_5f_cv_seeds.PNG" width="900" height="400">

|Modell| Lernrate | Grösse hL1 | Batchsize | Train Acc | Test Acc | Test Schätzfehler ($\sigma$) |
|----------|----------|----------|----------   |---------- | -------- |-------- |
|MLP-hl1-5f-cv (grün)   |1e-2      | 1024     | 32           | 0.6795       | 0.6139       | 0.086      |
|MLP-hl1-seeds (gelb)   |1e-2      | 1024     | 32           | 0.5648      | 0.5071       | 0.0088      |

---


**MLP mit 2 hidden Layers**  
Ein gängiger Ansatz ist die Grösse der Layer schrittweise zu verringern. In den folgenden Experimenten soll die Lernrate, die Grössen der hidden Layers und die Batchsize untersucht werden:

|Experiment| Lernrate | Grösse hL1 | Grösse hL2 |Batchsize |
|----------|----------|----------|----------|----------|
|1          | 1e-1, 1e-2, 1e-3, 1e-4, 1e-5 | 1024*                       | 256*  | 32*  |
|2          | 1e-2**                        | 1024, 2048  |  256*        | 32*        |
|3          | 1e-2**                        | 1024**                    | 64, 128, 256, 512  | 32* |
|4          | 1e-2**                        | 1024**                    | 512* | 4, 16, 32, 64, 128 |

Info:  
(\*) gewählter Startwert    
(\*\*) optimaler Wert    

In [ ]:
class MLPNet_hl2(nn.Module):
    def __init__(self, h_layer_sizes:list, input_size=3*32*32, act_fn=F.relu, dropout=0, 
                init_methode:str='kaiming_norm', output_size=10, bn=False):
        super(MLPNet_hl2, self).__init__()
        self.bn = bn

        self.linear1 = nn.Linear(input_size, h_layer_sizes[0])  # input.shape = (n, 3, 32, 32)
        self.bn1 = nn.BatchNorm1d(h_layer_sizes[0])
        self.linear2 = nn.Linear(h_layer_sizes[0], h_layer_sizes[1])
        self.bn2 = nn.BatchNorm1d(h_layer_sizes[1])
        self.linear3 = nn.Linear(h_layer_sizes[1], output_size)

        self.dropout = nn.Dropout(dropout)
        self.activation = act_fn 

        for linear_weight in [self.linear1.weight, self.linear2.weight]:          
            if init_methode == 'uniform':
                torch.nn.init.uniform_(linear_weight)
            if init_methode == 'xavier':
                torch.nn.init.xavier_uniform_(linear_weight)
            if init_methode == 'normal':
                torch.nn.init.normal_(linear_weight, mean=0, std=1)
            if init_methode == 'kaiming_norm':
                torch.nn.init.kaiming_normal_(linear_weight, nonlinearity='relu')

    def forward(self, x): # x.shape = (n, 3, 32, 32)
        x = x.view(-1, 3*32*32) # x.shape = (n, 3072)
        x = self.linear1(x)
        if self.bn: x = self.bn1(x)
        x = self.activation(x) 
        x = self.dropout(x)

        x = self.linear2(x)
        if self.bn: x = self.bn2(x)
        x = self.activation(x)
        x = self.dropout(x)

        x = self.linear3(x) 
        return x

In [ ]:
if False:
    from datetime import datetime

    # for lr in [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]:
    # for hl1_size in [1024, 2048]:
    # for hl2_size in [64, 128, 256, 512]:
    # for bt_size in [4, 16, 32, 64, 128]:
    for seed in [12, 16, 42, 64, 71]:
        # Hyperparameters  
        config = {
            "name": "MLP-hL2", 
            "epochs": 15,   
            "train_batch_size": 16, 
            "test_batch_size": 16,
            "dataset": "CIFAR-10",
            "lr": 1e-2, # default 1e-3
            "optimizer": 'SGD',
            "Regularisierung": 'None',  # 'None', 'L1', 'L2',
            "L1_lambda":0,
            "L2_weight_decay":0,
            "loss_func": 'CrossEntropyLoss',
            "loss_func": 'CrossEntropyLoss',
            "activation": "ReLU",
            "image_size": 32,
            "cross_validation": False,
            "n_folds": 5,
            "is_test_batch": False,
            "start_time": datetime.now().strftime("%d.%m.%Y_%H%M"),
            "num_workers": 0,
            "normalize":"zero_one",
            "init_w_method":"kaiming_norm",  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
            "hidden_layer_sizes": [1024, 512],
            "filter_sizes": [32, 64],
            "dense_layers": [512],
            "kernel_sizes": 3,
            "padding": 1,
            "stride": 1,
            "pool_sizes": [],
            "dropout": 0,
            "norm_mean": (0.5, 0.5, 0.5),
            "norm_std": (0.5, 0.5, 0.5),
            "num_classes": 10,
            "save_eval_image": True,
            "set_seed": seed  # [12, 16, 42, 64, 71]
        }

        # create model
        mlp_h2 = MLPNet_hl2(
            h_layer_sizes=config["hidden_layer_sizes"], 
            act_fn=F.relu,
            dropout=0,
            init_methode=config["init_w_method"],
            )

        cv_train_acc_mean, cv_test_acc_mean, _, _ = model_trainer(mlp_h2, config=config,
                        group=f'MLP_hL2_seed_val', # _5f_cv, _seed_val, _hl_size{}, _lr{}
                        tags=['MLP', 'seed val'], # 5fold cv, seed val, lr
                        print_info=False, plot_eval=True, sound=True, 
                        write_wandb=True
                        )


**Unterschiedliche Lernraten**

<img src="../01_Dokumentation/wandb_images/MLP_hl2_lr.PNG" width="900" height="400">

**Unterschiedliche Layergrössen Layer 1**

<img src="../01_Dokumentation/wandb_images/MLP_hl2_layer1_size.PNG" width="900" height="400">

**Unterschiedliche Layergrössen Layer 2**

<img src="../01_Dokumentation/wandb_images/MLP_hl2_layer2_size.PNG" width="900" height="400">

**Unterschiedliche Batchgrösse**

<img src="../01_Dokumentation/wandb_images/MLP_hl2_batch_size.PNG" width="900" height="400">


<div class="alert alert-block alert-success">

**Einfluss der Learningrate:**  
 Auch hier verzeichnet die Lernrate einen grossen Einfluss auf das Training. Lernraten von $1^{-5}$ oder $1^{-4}$ führen zu einem deutlich langsameren Training. Die lr von $1^{-2}$ erziehlte aus meiner Sicht das beste Ergebnis. Tiefere Training- und Validationslosses und die Accuracy schwankt ein wenig, aber es deutet noch kein Overfitting an.    


**Einfluss Grösse des Hidden Layers:**   
Das Modell umfasst nun zwei hidden Layer. Für den ersten Layer wurden zwei Grössen auspobiert 1024 und 2048. Beide Trainings führen zu sehr ähnlichen Ergebnissen im Loss und der Accuracy. Um einem Overfitting vorzubeugen wird die Layergrösse 1024 verwendet. Für den zweiten Layer wurden die Grössen 64, 128, 256, 512 ausprobiert. Die Grösse 512 erziehlte eine leicht höhere Accuracy und könnte mit einem längerem Training (mehr Epochen) evtl. das Modell weiter verbessern.

**Einfluss der Batchsize:**  
Die Batchgrösse sollte Einfluss auf die Stabilität des Trainings haben. Interessant ist, dass mit Batchsize 4 und 128 die Accuracy und der Validation Loss keine grossen Schwankungen zeigen. Bei Batchsize 4 wäre eine grössere Instabiltät erwartet worden. Batchsize 16 bietet die beste Accuracy bei einem stabilen Anstieg und den tiefesten Validationloss. Best Epoch wird bereits bei 10 gemessen was auf ein frühes Zeichen von Overfitting sein könnte. Deutlich spürbar war die Wahl der Batchsize bei der Dauer des Modelltrainings: Batchsize 4 dauerte 25 Minuten, Batchsize 16, 32, 64, 128 führten zu 9, 5, 4, 3 Minuten.

</div>

**MLP_hl2 Modell Evaluation**:  
Für die optimierten Parameter soll das Modell mit 5Fold-Cross-Validation und mit unterschiedlichen Seeds geprüft werden:  

*Bemerkung: Die Cross-Validation Option hat einen Fehler der noch nicht behoben ist. CV1 funktioniert, aber für die CV2-5 passen die Train- Testloader nicht überrein und führen zu inkorrekten Losses und Metriken. Dadurch wird die Performance stark überschätzt. Für den Schätzfehler sollen die Modelle zu den unterschiedlichen Seeds betrachtet werden.* 

<img src="../01_Dokumentation/wandb_images/MLP_hl2_5f_cv_seeds.PNG" width="900" height="400">

|Modell| Lernrate | Grösse hL1 | Grösse hL1 |Batchsize | Train Acc | Test Acc | Test Schätzfehler ($\sigma$) |
|------|----------|----------   |---------- |---------- |---------- | -------- |--------                    |
|MLP-hl2-5f-cv (--)   |1e-2      | 1024     | 512      | 16           | 0.8208       | 0.7026       | 0.1646      |
|MLP-hl2-seeds (-)   |1e-2      | 1024     | 512      | 16           | 0.6124      | 0.5310       | 0.0070      |

---

## **CNN** Hyperparameter Suche

Folgende wird ein einfaches CNN Modell erstellt. Es soll den Einfluss von verschiedenen Lernraten, Batchsize, Anzahl Filtern, Kernel-Grössen, Strides und Paddings untersucht werden:

**Ablauf:**  
1. verschiedene Lernraten werden untersucht
1. verschiedene Anzahl Filter werden untersucht
1. verschiedene Batchgrössen werden untersucht
1. verschiedene Kernelgrössen, Strides und Padding werden untersucht


---

**Simple CNN**  

|Experiment| Lernrate | Grösse Filter1 |  Grösse Filter2 |Batchsize |Kernel-Grösse | Stride | Padding |
|---|----------|----------|----------|----------|----------|----------|----------|
|1| 1e-1, 1e-2, 1e-3, 1e-4, 1e-5 | 32*  | 64*  | 32*  | 3*  | 1*  | 1*  |
|2| 1e-2** | 16, 32, 64 | 64* |  32* | 3*  | 1*  | 1*  |
|3| 1e-2** | 32**  | 32, 64, 96  | 32*  | 3*  | 1*  | 1*  |
|4| 1e-2** | 32**  | 64**  | 8, 16, 32, 64  | 3*  | 1*  | 1*  |
|5| 1e-2** | 32**  | 64**  | 32**  | 3, 5, 7  | 1*  | 1*  |
|6| 1e-2** | 32**  | 64**  | 32**  | 3** | 1, 2, 3  | 1*  |
|7| 1e-2** | 32**  | 64**  | 32**  | 3** | 1**  | 0, 1, 2  |

Info:  
(\*) gewählter Startwert    
(\*\*) optimaler Wert    

Dimensionsberechnungen für Covolution Layers:  
Um die CNN Klasse variable zu gestalten, müssen die Dimensionen anhand der Hyperparameter berechnet werden
$$Output\ Grösse = \frac{Input\ Grösse + (2 \cdot padding - Kernel\ Grösse)}{stride} + 1 $$

In [ ]:
def compute_conv_dim(input_size, kernel_size, padding, stride, pooling=True):
    conv_dim = int((input_size + 2 * padding - kernel_size) / stride + 1)

    if pooling: conv_dim = conv_dim / 2
    return conv_dim

# Berechnungstest
input_dim_bild = 32  # 32x32 pixel cifar-10
kernel_size = 3  # 3x3
padding = 1
stride = 1
conv2_out_channel = 64

print(f'{input_dim_bild=}')  
output_dim_conv1 = compute_conv_dim(input_dim_bild, kernel_size, padding, stride, pooling=True)
print(f'{output_dim_conv1=}') 

output_dim_conv2 = compute_conv_dim(output_dim_conv1, kernel_size, padding, stride, pooling=True)
print(f'{output_dim_conv2=}') 

input_fc1 = output_dim_conv2 * output_dim_conv2 * conv2_out_channel
print(f'{input_fc1=} = {output_dim_conv2} x {output_dim_conv2} x {conv2_out_channel}')

In [ ]:
# Modell Class
class SimpleCNN(nn.Module):
    def __init__(self, input_ch=3, act_fn=F.relu, filter_size:list=[32, 64], dlayers:list=[512], num_class=10,
                kernel_size=3, padding=1, stride=1, dropout_ch=0, bn=False, reg='L1', input_image=32):
        super(SimpleCNN, self).__init__()
        self.bn = bn
        self.reg = reg

        self.conv1 = nn.Conv2d(in_channels=input_ch, out_channels=filter_size[0], 
                            kernel_size=kernel_size, padding=padding, stride=stride)
        self.conv2 = nn.Conv2d(in_channels=filter_size[0], out_channels=filter_size[1], 
                            kernel_size=kernel_size, padding=padding, stride=stride)      
                
        self.dim1 = self.compute_conv_dim(input_image, kernel_size, padding, stride)
        self.dim2 = self.compute_conv_dim(self.dim1, kernel_size, padding, stride)
        
        self.fc1 = nn.Linear(self.dim2 * self.dim2 * filter_size[1], dlayers[0])
        self.fc2 = nn.Linear(dlayers[0], num_class)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.activation = act_fn 

        # Dropout for convolution layers
        self.dropout2D = nn.Dropout2d(dropout_ch)
        # Dropout for dense layers
        self.dropout = nn.Dropout(dropout_ch)

        # Batch normalization layers for convolution layers
        self.bn1 = nn.BatchNorm2d(filter_size[0])
        self.bn2 = nn.BatchNorm2d(filter_size[1])
        
        # Batch normalization layer for dense layer
        self.bn_fc1 = nn.BatchNorm1d(dlayers[0])
        self.dim = []

    @staticmethod
    def compute_conv_dim(input_size, kernel_size, padding, stride, pooling=True):
        conv_dim = int((input_size + 2 * padding - kernel_size) / stride + 1)
        if pooling: conv_dim = int(conv_dim / 2)
        return conv_dim

    def forward(self, x):
        x = self.conv1(x)
        if self.bn: x = self.bn1(x)
        x = self.activation(x)
        x = self.pool(x)
        x = self.dropout(x)
        
        x = self.conv2(x)
        if self.bn: x = self.bn2(x)
        x = self.activation(x)
        x = self.pool(x)
        x = self.dropout(x)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        if self.bn: x = self.bn_fc1(x)
        x = self.activation(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        return x

In [ ]:
if False:
    from datetime import datetime

    # for lr in [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]: 
    # for f1_size in [16, 32, 64]:
    # for f2_size in [32, 64, 96]:
    # for bt_size in [8, 16, 32, 64]:
    # for kernel_size in [3, 5, 7]:
    # for stride in [1, 2, 3]:
    # for padding in [0, 1, 2]:  
    for seed in [12, 16, 42, 64, 71]:
        # Hyperparameters  
        config = {
            "name": "CNN-simple", 
            "epochs": 20,   
            "train_batch_size": 32, 
            "test_batch_size": 32,
            "dataset": "CIFAR-10",
            "lr": 1e-2, # default 1e-3
            "optimizer": 'SGD',
            "Regularisierung": 'None',  # 'None', 'L1', 'L2',
            "L1_lambda":0,
            "L2_weight_decay":0,
            "loss_func": 'CrossEntropyLoss',
            "activation": "ReLU",
            "image_size": 32,
            "cross_validation": False,
            "n_folds": 5,
            "is_test_batch": False,
            "start_time": datetime.now().strftime("%d.%m.%Y_%H%M"),
            "num_workers": 0,
            "normalize":"zero_one",
            "init_w_method":"kaiming_norm",  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
            "hidden_layer_sizes": [1024, 256],
            "filter_sizes": [32, 64],
            "dense_layers": [512],
            "kernel_sizes": 3,
            "padding": 2,
            "stride": 1,
            "pool_sizes": [],
            "dropout": 0,
            "norm_mean": (0.5, 0.5, 0.5),
            "norm_std": (0.5, 0.5, 0.5),
            "num_classes": 10,
            "save_eval_image": True,
            "set_seed": seed  # [12, 16, 42, 64, 71]
        }

        # create model
        cnn_simple = SimpleCNN(
            filter_size=config["filter_sizes"],
            dlayers=config["dense_layers"],
            act_fn=F.relu,
            kernel_size=config["kernel_sizes"],
            padding=config["padding"],
            stride=config["stride"],
            dropout_ch=0,
            )

        cv_train_acc_mean, cv_test_acc_mean, _, _ = model_trainer(cnn_simple, config=config,
                        group=f'CNN_simple_seed_val', # _5f_cv, _seed_val, _hl_size{}, _lr{}
                        tags=['CNN', 'seed val'], # 5fold cv, seed val, lr
                        print_info=False, plot_eval=True, sound=True, 
                        write_wandb=True
                        )


**Unterschiedliche Lernrate**

<img src="../01_Dokumentation/wandb_images/CNN_simple_lr.PNG" width="900" height="400">

**Unterschiedliche Anzahl Filter**

<img src="../01_Dokumentation/wandb_images/CNN_simple_num_filters.PNG" width="900" height="400">

**Unterschiedliche Batchgrössen**

<img src="../01_Dokumentation/wandb_images/CNN_simple_bt_sizes.PNG" width="900" height="400">

**Unterschiedliche Kernel Grössen**

<img src="../01_Dokumentation/wandb_images/CNN_simple_kernel_sizes.PNG" width="900" height="400">

**Unterschiedliche Strides und Paddings**

<img src="../01_Dokumentation/wandb_images/CNN_simple_strides_paddings.PNG" width="900" height="400">


<div class="alert alert-block alert-success">

**Einfluss der Learningrate:**  
Um die Unterschiede der Lernraten besser zu zeigen wurden Modelle für 20 Epochen trainiert. Die Lernraten $1^{-5}$ und $1^{-4}$ sind zu tief und führen zu keinem Training. Die Lernraten $1^{-1}$ und $1^{-2}$ führen zu einer sehr ähnlichen Test Accuracy, wobei der Trainingsprozess sehr verschieden ist. Mit lr $1^{-1}$ steigt der Validationloss ab Epoche 4 stark an, auch die Trainings Accuracy ist sehr hoch. Das Modell trainiert sehr schnell und sollte nach ein paar Epochen beendet werden. Die lr $1^{-2}$ wurde ausgewählt da sie einen stabileren Eindruck vermittelt.


**Einfluss der Filter Grösse:**   
Mit mehr Filter können mehr Merkmale erkannt und komplexere Aufgaben gelöst werden. Für die 32x32 Bilder von Cifar-10 wurde eine unterschiedliche Anzahl von Filtern getestet. Für den ersten CNN-Layer wurden die Filtergrösse 16, 32, 64 auspobiert, für den zweiten Layer die Grössen 32, 64, 96. Anhand der Accuracy und der Losswerten konnte keinen markanten Einfluss auf die Modelloptimerung festgestellt werden. Für das weitere Training wurden die Grössen 32 und 64 für die beiden Layer verwendet. Auf die Trainingszeiten hatte die Filtergrösse auch keinen Einfluss. Bei klompexeren Daten/Aufgaben und mehr CNN-Layer, könnte es sich evtl. dennoch lohnen die Filtergrösse zu untersuchen.


**Einfluss der Batchgrössen:**  
Die Batchgrösse hängt auch stark vom GPU-Speicher ab, eine RTX-2070 GPU mit 8GB war bei einer Batchsize von 64 zu 70% ausgelastet.
Ähnliche Erkentnisse wie bei den MLP Modellen sehen wir verstärkt für die CNN Modelle. Mit einer Batchsize von 8 kann das Training viel früher gestoppt werden. Auch hier verzeichnet eine tiefe Batchsize eine lange Trainingsdauer für 20 Epochen. Da das Training aber viel schneller lernt, werden keine 20 Epochen benötigt. Für mehr Stabilität wurde dennoch eine Batchsize von 16 gewählt. Hier können später auch die Regularisierungs Methoden gut ausprobiert werden.


**Einfluss der Kernel Grösse:**  
Kleiner Kernel von 1x1 oder 3x3 sind besser geeignet um lokale Informationen zu erfassen, Kernel mit 5x5 und 7x7 fangen eher globale Muster auf. Die Kernel Grössen 3,5,7 wurden ausprobiert. Auf die Bilder von Cifar-10 konnte keinen wesentlichen Einfluss bemerkt werden. Der Validationloss für die Grössen 5,7 ist gegen Ende höher, was auf eine schnelleres Overfitting durch mehr globale Informationen hindeuten könnte. 


**Einfluss von Stride und Padding:**  
**Stride** bezeichnet die Schrittweite, mit der der Kernel über das Bild fährt. Bei einem Wert von 1 überlappt sich der Filter, bei grösseren Werten kann das Netzwerk schneller grössere Muster erkennen. Ein hoher Stride kann auch als Pooling Ersatz dienen, also die Bilddimension reduzieren. Bisher habe ich in Modellen nur Stridewerte von 1 gesehen, darum soll hier untersucht werden welchen Einfluss von grösseren Werten erkennbar ist. Ein Stridewert von 1 zeigte deutlich bessere Modell Performance. Dadurch dass die 32x32 Bilder nicht sehr gross sind, werden wohl eher kleine Muster benötigt. Interessant wäre es, den Pooling Layer durch einen höheren Stride zu ersetzen.   
**Padding** fügt rund um das Bild weitere Pixel hinzu (mit Pytorch default Pixelwert ist 0 - Zero Padding), dies kann verwendet werden um die Bilddimension nach einer Convolution beizubehalten. Bei einem Padding > 0 kann zudem der ganze Filter auch am Rand des Bilder angewendet werden und Muster erkennen. Es wurde Padding zu den Werten 0, 1, 2 untersucht (Kernelgrösse bei 3x3 festgelegt, dadurch Padding bei max 2). Nicht sehr markant aber das Training und die Accuracy profitieren von Padding = 2. Zu beachten ist, dass die Objekte in den Bildern wohl eher zentriert sind.

</div>

**CNN Modell Evaluation**:  
Für die optimierten Parameter soll das Modell mit 5Fold-Cross-Validation und mit unterschiedlichen Seeds geprüft werden:

*Bemerkung: Die Cross-Validation Option hat einen Fehler der noch nicht behoben ist. CV1 funktioniert, aber für die CV2-5 passen die Train- Testloader nicht überrein und führen zu inkorrekten Losses und Metriken. Dadurch wird die Performance stark überschätzt. Für den Schätzfehler sollen die Modelle zu den unterschiedlichen Seeds betrachtet werden.* 

<img src="../01_Dokumentation/wandb_images/CNN_simple_5f_cv_seeds.PNG" width="900" height="400">

|Modell| Lernrate | Grösse Filter1 | Grösse Filter2 |Batchsize | Kernel, Stride, Padding |Train Acc | Test Acc | Test Schätzfehler ($\sigma$) |
|------|----------|----------   |---------- |---------- |---------- | --------          |--------        |--------                    |
|CNN-simple-5f-cv   |1e-2      | 32     | 64      | 32      |  3,1,2    | 0.9671       | 0.8978       | 0.139      |
|CNN-simple-seeds   |1e-2      | 32     | 64      | 32      |  3,1,2    | 0.8453      | 0.6900       | 0.009      |


<div class="alert alert-block alert-success">

### Fazit Modelkomplexität
Die CNN Modelle sind den MLP Modellen überlegen. Die spezielle Verarbeitung mit den Convolutions kann Bilder besser verarbeiten und lokale Muster besser erkennen.

</div>

---

<div class="alert alert-block alert-info">

**2. Nutzen der Regularisierung**  
Ziehe nun verschiedene Regularisierungsmethoden bei den MLP Layern in Betracht:  
a. L1/L2 Weight Penalty  
b. Dropout

Evaluiere den Nutzen der Regularisierung, auch unter Berücksichtigung verschiedener Regularisierungsstärken. Beschreibe auch kurz, was allgemein das Ziel von Regularisierungsmethoden ist (Regularisierung im Allgemeinen, sowie auch Idee der einzelnen Methoden). Inwiefern wird dieses Ziel im gegebenen Fall erreicht?
    
</div>

<div class="alert alert-block alert-success">

Das allgemeine Ziel der Regularisierung ist eine Generalisierung des Modells. Spezfische Deep Learning ist vom Overfitting Problem betroffen, die Regularisierung schränkt die Komplexität des Modells und soll ein OVerfitting eingrenzen/verhindern.

**L1**  
Die Lasso-Regularisierung fügt der Lossfunktion die absolute Summe der Gewichte hinzu. Der $ \lambda$-Wert  ist der Regularisierungskoeffizient. Mit L1 werden die Gewichte eher auf Null gesetzt, was idealer Weise weniger relevante Merkmale gleich entfernt.

**L2**  
Die Ridge-Regularisierung lässt die Gewichte während dem Training schrumpfen. Die quadrierten Gewichtswerte werden mit 'weight-decay' skalliert und zur Lossfunktion hinzugefügt. In Pytorch steht dafür der Hyperparameter 'weight-decay' zur Verfügung. Bei jeder Epoche werden die Gewichte tendenziell gegen 0 gezogen.

**Dropout**  
**MLP**: Bei jedem Trainingsschritt werden zufällig Neuronen deaktiviert (einzelne Elemente in einem Tensor). Das Netzwerk kann sich so nicht auf einzelnen Neuronen verlassen.  
**CNN**: Bei Dropout_2D werden ganze Känale oder Feature-Maps deaktiviert. Das Netzwerk kann sich so nicht auf einzelne Features verlassen.

**Early Stopping**  
Das Modelltraining wird beendet sobald die Performance auf dem Validierungsdatensatz nicht verbessert werden kann. 

</div>

## **MLP** Regularisierung

Folgende wird zum MLP Modell mit 2 Hidden Layers den Einsatz von Regularisierungen getestet. Es soll der Einfluss der `L1`, `L2` Regularisierung sowie von `Dropout` untersucht werden.

**Ablauf:**  
1. Modell ohne Regularisierung mit längerer Trainingsdauer
1. L1-Regularisierung zu verschiedenen L1-Lambda Werten
1. L2-Regularisierung zu verschiedenen Weight-Decay Werten
1. Dropout mit verschiedenen Wahrscheinlichkeiten


|Experiment| L1-Lambda | L2-Weight Decay |  Dropout  
|---|----------|----------|----------|
|1| 0| 0  | 0  |
|2| 1e-5, 1e-4  | 0  | 0 | 
|3| 0 | 1e-4, 1e-3, 1e-2|  0 | 
|4| 0 | 0 | 0.2, 0.4, 0.6, 0.8  | 


---

In [ ]:
if False:
    from datetime import datetime

    # for no_reg in [1]:
    # for L1_lambda in [1e-5, 1e-4]:  
    # for L2_w_decay in [1e-4, 1e-3, 1e-2]:
    for dropout_p in [0.2, 0.4, 0.6, 0.8]:
        # Hyperparameters  
        config = {
            "name": "MLP-hL2", 
            "epochs": 25,   
            "train_batch_size": 16, 
            "test_batch_size": 16,
            "dataset": "CIFAR-10",
            "lr": 1e-2, # default 1e-3
            "optimizer": 'SGD',
            "Regularisierung": 'None',  # 'None', 'L1', 'L2',
            "L1_lambda": 0,
            "L2_weight_decay": 0,
            "loss_func": 'CrossEntropyLoss',
            "loss_func": 'CrossEntropyLoss',
            "activation": "ReLU",
            "image_size": 32,
            "cross_validation": False,
            "n_folds": 5,
            "is_test_batch": False,
            "start_time": datetime.now().strftime("%d.%m.%Y_%H%M"),
            "num_workers": 0,
            "normalize":"zero_one",
            "init_w_method":"kaiming_norm",  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
            "hidden_layer_sizes": [1024, 512],
            "filter_sizes": [32, 64],
            "dense_layers": [512],
            "kernel_sizes": 3,
            "padding": 1,
            "stride": 1,
            "pool_sizes": [],
            "dropout": dropout_p,
            "norm_mean": (0.5, 0.5, 0.5),
            "norm_std": (0.5, 0.5, 0.5),
            "num_classes": 10,
            "save_eval_image": True,
            "set_seed": 42 
        }

        # create model
        mlp_h2 = MLPNet_hl2(
            h_layer_sizes=config["hidden_layer_sizes"], 
            act_fn=F.relu,
            dropout=config['dropout'],
            init_methode=config["init_w_method"],
            )

        cv_train_acc_mean, cv_test_acc_mean, _, _ = model_trainer(mlp_h2, config=config,
                        group=f'MLP_hL2_drop_{dropout_p}', # _5f_cv, _seed_val, _hl_size{}, _lr{}
                        tags=['MLP', 'dropout'], # 5fold cv, seed val, lr
                        print_info=False, plot_eval=True, sound=True, 
                        write_wandb=True
                        )


**Unterschiedliche L1-Lambda Werte**

<img src="../01_Dokumentation/wandb_images/MLP_hl2_reg_l1.PNG" width="900" height="400">

**Unterschiedliche L2-Weight Decay Werte**

<img src="../01_Dokumentation/wandb_images/MLP_hl2_reg_L2.PNG" width="900" height="400">

**Unterschiedliche Dropout**

<img src="../01_Dokumentation/wandb_images/MLP_hl2_reg_dropout.PNG" width="900" height="400">

<div class="alert alert-block alert-success">

**Unterschiedliche L1-Lambda Werte**  
Verglichen wurde das MLP mit zwei hidden Layer ohne und mit Regularisierung zu unterschiedlichen $\lambda$-Werten. Bei $\lambda=1^{-5}$ noch wenig dann bei $\lambda=1^{-4}$ wird das Training erheblich beeinflusst. Ohne Regularisierung findet ein Overfitting ab Epoche 16 statt. Der Einfluss von L1 kann vor allem im Trainingsloss und der Trainings Accuracy gesehen werden. Das Modell mit $\lambda=1^{-4}$ zeigt eine tiefere Test Accuracy, aber dafür ist sie näher an der Training Accurcy, was eher für ein generalisiertes Modell steht. Um Trainingszeit zu sparen, würde das Einsetzen von 'early-stopping' auch Sinn machen.

**Unterschiedliche L2-Weight Decay Werte**   
Der Wert $1^{-2}$ Regularisiert das Modell zu stark. Der Wert $1^{-3}$ beeinflusst die Test Accuracy nur sehr wenig, reguliert aber den Trainingsloss und die Trainings Accuracy. Auch hier würde wohl 'early-stopping' einen ähnlichen Effekt erzielen können.

**Unterschiedliche Dropout**  
Verschiedene Dropout Wahrscheinlichkeiten wurden getestet, 0.2, 0.4, 0.6 und 0.8. Auch hier gut ersichtlich den Einfluss auf die Trainings Accuracy. Ein nutzvoller Wert scheint bei $dropout=0.4$ zu liegen.

</div>

## **CNN** Regularisierung

Folgende wird zum CNN simple Modell den Einsatz von Regularisierungen getestet. Es soll der Einfluss der `L1`, `L2` Regularisierung sowie von `Dropout` untersucht werden.

**Ablauf:**  
1. Modell ohne Regularisierung mit längerer Trainingsdauer
1. L1-Regularisierung zu verschiedenen L1-Lambda Werten
1. L2-Regularisierung zu verschiedenen Weight-Decay Werten
1. Dropout mit verschiedenen Wahrscheinlichkeiten


|Experiment| L1-Lambda | L2-Weight Decay |  Dropout |
|---|----------|----------|----------|
|1| 0| 0  | 0  | 
|2| 1e-5, 1e-4 | 0  | 0 
|3| 0 | 1e-5, 1e-4, 1e-3, 1e-2|  0 
|4| 0 | 0  | 0.2, 0.4, 0.6  

---

In [ ]:
if False:
    from datetime import datetime

    # for no_reg in [1]:
    # for L1_lambda in [1e-5, 1e-4, 1e-3]:
    # for L2_w_decay in [1e-4, 1e-3, 1e-2]:
    for dropout_p in [0.2, 0.4, 0.6, 0.8]:
        # Hyperparameters  
        config = {
            "name": "CNN-simple", 
            "epochs": 30,   
            "train_batch_size": 32, 
            "test_batch_size": 32,
            "dataset": "CIFAR-10",
            "lr": 1e-2, # default 1e-3
            "optimizer": 'SGD',
            "Regularisierung": 'None',  # 'None', 'L1', 'L2',
            "L1_lambda":0,
            "L2_weight_decay":0,
            "loss_func": 'CrossEntropyLoss',
            "activation": "ReLU",
            "image_size": 32,
            "cross_validation": False,
            "n_folds": 5,
            "is_test_batch": False,
            "start_time": datetime.now().strftime("%d.%m.%Y_%H%M"),
            "num_workers": 0,
            "normalize":"zero_one",
            "init_w_method":"kaiming_norm",  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
            "hidden_layer_sizes": [1024, 256],
            "filter_sizes": [32, 64],
            "dense_layers": [512],
            "kernel_sizes": 3,
            "padding": 2,
            "stride": 1,
            "pool_sizes": [],
            "dropout": dropout_p,
            "norm_mean": (0.5, 0.5, 0.5),
            "norm_std": (0.5, 0.5, 0.5),
            "num_classes": 10,
            "save_eval_image": True,
            "set_seed": 42  # [12, 16, 42, 64, 71]
        }

        # create model
        cnn_simple = SimpleCNN(
            filter_size=config["filter_sizes"],
            dlayers=config["dense_layers"],
            act_fn=F.relu,
            kernel_size=config["kernel_sizes"],
            padding=config["padding"],
            stride=config["stride"],
            dropout_ch=config["dropout"],
            )

        cv_train_acc_mean, cv_test_acc_mean, _, _ = model_trainer(cnn_simple, config=config,
                        group=f'CNN_simple_drop_{dropout_p}', # _5f_cv, _seed_val, _hl_size{}, _lr{}
                        tags=['CNN', 'dropout reg'], # 5fold cv, seed val, lr
                        print_info=False, plot_eval=True, sound=True, 
                        write_wandb=True
                        )


**Unterschiedliche L1-Lambda Werte**

<img src="../01_Dokumentation/wandb_images/CNN_simple_reg_L1.PNG" width="900" height="400">

**Unterschiedliche L2-Weight Decay Werte**

<img src="../01_Dokumentation/wandb_images/CNN_simple_reg_L2.PNG" width="900" height="400">

**Unterschiedliche Dropout**

<img src="../01_Dokumentation/wandb_images/CNN_simple_reg_dropout.PNG" width="900" height="400">

<div class="alert alert-block alert-success">

**Unterschiedliche L1-Lambda Werte**  
Der $\lambda$-Wert für L1 ist auch hier sehr empfindlich und muss sorgfälltig gewählt werden. Neben einer Regularisierung von $\lambda=1^{-4}$ könnte zusätzlich auch 'early stopping' vewendet werden.

**Unterschiedliche L2-Weight Decay Werte**  
Ein optimaler Wert scheint sich um $1^{-2}$ herum zu finden.

**Unterschiedliche Dropout Wahrscheinlichkeit**  
Verschiedene Dropout Wahrscheinlichkeiten wurden getestet, 0.2, 0.4, 0.6 und 0.8. Auch hier gut ersichtlich den Einfluss auf die Trainings Accuracy. Ein nutzvoller Wert scheint hier bei $dropout=0.3$ zu liegen.  
*Bemerkung: Der Dropout Parameter setzt die dropout sowie droupout_2d Rate*


</div>

---

<div class="alert alert-block alert-info">

**3. Nutzen von Batchnorm BN (ohne REG, mit SGD)**  
Evaluiere, ob Batchnorm etwas bringt. Beschreibe kurz, was die Idee von BN ist, wozu es helfen soll.
    
</div>    

<div class="alert alert-block alert-success">

Batch Normalization (BN) ermöglicht ein schnelleres und stabileres Training von tiefen Netzwerken. Die Aktivierung innerhalb eines Batches haben so den Mittelwert 0 und eine Standardabweichung von 1. Während dem Training verschieben sich normalerweise die Verteilungen der Aktivierungen, BN soll das verhindern. BN macht das Netzwerk auch weniger empfindlich gegenüber der Initialisierung der Gewichte und hat leichte regularisierende Effekte.

</div>

## **MLP** und **CNN** Batchnorm

Folgend wird zum MLP und CNN Modell den Einsatz von Batchnormalisierung getestet. 

**Ablauf:**  
1. Modelle ohne Batchnormalisierung 
1. Modelle mit Batchnormalisierung 


|Modell| Batchnorm |
|---|----------|
|MLP-h2l| Aus| 
|MLP-h2l|  Ein | 
|CNN-simple| Aus | 
|CNN-simple| Ein | 

---

In [ ]:
# MLP
if False:
    from datetime import datetime

    for bn in [False, True]:
        # Hyperparameters  
        config = {
            "name": "MLP-hL2", 
            "epochs": 25,   
            "train_batch_size": 16, 
            "test_batch_size": 16,
            "dataset": "CIFAR-10",
            "lr": 1e-2, # default 1e-3
            "optimizer": 'SGD',
            "Regularisierung": 'None',  # 'None', 'L1', 'L2',
            "L1_lambda": 0,
            "L2_weight_decay": 0,
            "batchnorm": bn,
            "loss_func": 'CrossEntropyLoss',
            "loss_func": 'CrossEntropyLoss',
            "activation": "ReLU",
            "image_size": 32,
            "cross_validation": False,
            "n_folds": 5,
            "is_test_batch": False,
            "start_time": datetime.now().strftime("%d.%m.%Y_%H%M"),
            "num_workers": 0,
            "normalize":"zero_one",
            "init_w_method":"kaiming_norm",  #['uniform', 'xavier', 'normal' ,'kaiming_norm']
            "hidden_layer_sizes": [1024, 512],
            "filter_sizes": [32, 64],
            "dense_layers": [512],
            "kernel_sizes": 3,
            "padding": 1,
            "stride": 1,
            "pool_sizes": [],
            "dropout": 0,
            "norm_mean": (0.5, 0.5, 0.5),
            "norm_std": (0.5, 0.5, 0.5),
            "num_classes": 10,
            "save_eval_image": True,
            "set_seed": 42 
        }

        # create model
        mlp_h2 = MLPNet_hl2(
            h_layer_sizes=config["hidden_layer_sizes"], 
            act_fn=F.relu,
            dropout=config['dropout'],
            init_methode=config["init_w_method"],
            bn=config["batchnorm"]
            )

        cv_train_acc_mean, cv_test_acc_mean, _, _ = model_trainer(
            mlp_h2, config=config,
            group=f'MLP_hL2_bn_{bn}', # _5f_cv, _seed_val, _hl_size{}, _lr{}
            tags=['MLP', 'batchnorm'], # 5fold cv, seed val, lr
            print_info=False, plot_eval=True, sound=True, 
            write_wandb=True
            )


In [ ]:
# CNN
if False:
    from datetime import datetime

    for bn in [False, True]:
        # Hyperparameters  
        config = {
            "name": "CNN-simple", 
            "epochs": 30,   
            "train_batch_size": 32, 
            "test_batch_size": 32,
            "dataset": "CIFAR-10",
            "lr": 1e-2, # default 1e-3
            "optimizer": 'SGD',
            "Regularisierung": 'None',  # 'None', 'L1', 'L2',
            "L1_lambda":0,
            "L2_weight_decay":0,
            "batchnorm": bn,
            "loss_func": 'CrossEntropyLoss',
            "activation": "ReLU",
            "image_size": 32,
            "cross_validation": False,
            "n_folds": 5,
            "is_test_batch": False,
            "start_time": datetime.now().strftime("%d.%m.%Y_%H%M"),
            "num_workers": 0,
            "normalize":"zero_one",
            "init_w_method":"kaiming_norm",  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
            "hidden_layer_sizes": [1024, 256],
            "filter_sizes": [32, 64],
            "dense_layers": [512],
            "kernel_sizes": 3,
            "padding": 2,
            "stride": 1,
            "pool_sizes": [],
            "dropout": 0,
            "norm_mean": (0.5, 0.5, 0.5),
            "norm_std": (0.5, 0.5, 0.5),
            "num_classes": 10,
            "save_eval_image": True,
            "set_seed": 42  # [12, 16, 42, 64, 71]
        }

        # create model
        cnn_simple = SimpleCNN(
            filter_size=config["filter_sizes"],
            dlayers=config["dense_layers"],
            act_fn=F.relu,
            kernel_size=config["kernel_sizes"],
            padding=config["padding"],
            stride=config["stride"],
            dropout_ch=config["dropout"],
            bn=config["batchnorm"]
            )

        cv_train_acc_mean, cv_test_acc_mean, _, _ = model_trainer(
            cnn_simple, config=config,
            group=f'CNN__simple_bn_{bn}', # _5f_cv, _seed_val, _hl_size{}, _lr{}
            tags=['CNN', 'batchnorm'], # 5fold cv, seed val, lr
            print_info=False, plot_eval=True, sound=True, 
            write_wandb=True
            )


**Verwednung Batchnorm MLP und CNN**

<img src="../01_Dokumentation/wandb_images/MLP_CNN_batchnorm.PNG" width="800" height="400">


<div class="alert alert-block alert-success">

**Nutzen von Batchnorm für MLP und CNN** (ohne REG, mit SGD)  
**MLP**: Das Modell Training findet viel schneller statt (Trainingsloss). Die Anzahl Epochen kann starch verringert werden, ein Overfitting findet mit BN bereits ab Epoche 6 statt.  
**CNN**: Auch findet das Modell Training sehr viel schneller statt und die Trainingsdauer kann mit BN stark reduziert werden (Achtung Overfitting). Eine Verbesserung der Stabiltät konnte nicht beobachtet werden.  

In Verbindung mit 'early-stopping' kann eine BN eine erhebliche Steigerung in der Trainingszeit bewirken.

</div>

---

<div class="alert alert-block alert-info">

**4. Nutzen von Adam (ohne BN, ohne / mit REG)**   
Evaluiere, ob Du mit Adam bessere Resultate erzielen kannst.
    
</div>    

<div class="alert alert-block alert-success">

**SGD** steht 'Stochastic Gradient Descent' und besitzt eine feste Lernrate. **Adam** 'Adaptive Moment Estimation' passt die Lernrate individuell an und ist bekannt für seine schnelle Konvergenz am Anfang des Modelltrainings.  

</div>

## **MLP** und **CNN** Optimizer Adam

Folgend wird zum MLP und CNN Modell den Einsatz des Adam Optimizers getestet. 

**Ablauf:**  
1. Modell mit SGD und mit/ohne Regularisierung
1. Modell mit Adam und mit/ohne Regularisierung


|Modell| Optimizer | Regularisierung | 
|---|----------|----------|
|MLP-h2l| SGD|  ohne  |
|MLP-h2l| SGD|  mit L1/dropout  | 
|MLP-h2l|  Adam | ohne  | 
|MLP-h2l|  Adam | mit L1/dropout  | 
|CNN-simple| SGD |   ohne | 
|CNN-simple| SGD |  mit L2/dropout |
|CNN-simple| Adam |  ohne |
|CNN-simple| Adam |  mit L2/dropout | 

---

In [ ]:
# MLP
if False:
    from datetime import datetime

    for reg in ['None', 'L1']:  # 'None', 'L1'
    # for lr in [1e-3, 1e-4, 1e-5]:
        # Hyperparameters  
        config = {
            "name": "MLP-hL2", 
            "epochs": 18,   
            "train_batch_size": 16, 
            "test_batch_size": 16,
            "dataset": "CIFAR-10",
            "lr": 1e-4, # default 1e-3
            "optimizer": 'Adam',  #'SGD', 'Adam'
            "Regularisierung": reg,  # 'None', 'L1', 'L2',
            "L1_lambda": 1e-4,
            "L2_weight_decay": 1e-3,
            "batchnorm": False,
            "loss_func": 'CrossEntropyLoss',
            "activation": "ReLU",
            "image_size": 32,
            "cross_validation": False,
            "n_folds": 5,
            "is_test_batch": False,
            "start_time": datetime.now().strftime("%d.%m.%Y_%H%M"),
            "num_workers": 0,
            "normalize":"zero_one",
            "init_w_method":"kaiming_norm",  #['uniform', 'xavier', 'normal' ,'kaiming_norm']
            "hidden_layer_sizes": [1024, 512],
            "filter_sizes": [32, 64],
            "dense_layers": [512],
            "kernel_sizes": 3,
            "padding": 1,
            "stride": 1,
            "pool_sizes": [],
            "dropout": 0.2,
            "norm_mean": (0.5, 0.5, 0.5),
            "norm_std": (0.5, 0.5, 0.5),
            "num_classes": 10,
            "save_eval_image": True,
            "set_seed": 42 
        }

        # create model
        mlp_h2 = MLPNet_hl2(
            h_layer_sizes=config["hidden_layer_sizes"], 
            act_fn=F.relu,
            dropout=config['dropout'],
            init_methode=config["init_w_method"],
            bn=config["batchnorm"]
            )

        cv_train_acc_mean, cv_test_acc_mean, _, _ = model_trainer(
            mlp_h2, config=config,
            group=f'MLP_hL2_adam_reg_{reg}', # _5f_cv, _seed_val, _hl_size{}, _lr{}
            tags=['MLP', 'adam'], # 5fold cv, seed val, lr
            print_info=False, plot_eval=True, sound=True, 
            write_wandb=True
            )


In [ ]:
# CNN
if False:
    from datetime import datetime

    for reg in ['None', 'L2']:
    # for lr in [1e-5, 1e-4, 1e-3, 1e-2]:
        # Hyperparameters  
        config = {
            "name": "CNN-simple", 
            "epochs": 18,   
            "train_batch_size": 32, 
            "test_batch_size": 32,
            "dataset": "CIFAR-10",
            "lr": 1e-4, # default 1e-3
            "optimizer": 'Adam',  #'SGD', 'Adam'
            "Regularisierung": reg,  # 'None', 'L1', 'L2',
            "L1_lambda":1e-4,
            "L2_weight_decay":1e-3,
            "batchnorm": False,
            "loss_func": 'CrossEntropyLoss',
            "activation": "ReLU",
            "image_size": 32,
            "cross_validation": False,
            "n_folds": 5,
            "is_test_batch": False,
            "start_time": datetime.now().strftime("%d.%m.%Y_%H%M"),
            "num_workers": 0,
            "normalize":"zero_one",
            "init_w_method":"kaiming_norm",  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
            "hidden_layer_sizes": [1024, 256],
            "filter_sizes": [32, 64],
            "dense_layers": [512],
            "kernel_sizes": 3,
            "padding": 2,
            "stride": 1,
            "pool_sizes": [],
            "dropout": 0.2,
            "norm_mean": (0.5, 0.5, 0.5),
            "norm_std": (0.5, 0.5, 0.5),
            "num_classes": 10,
            "save_eval_image": True,
            "set_seed": 42  # [12, 16, 42, 64, 71]
        }

        # create model
        cnn_simple = SimpleCNN(
            filter_size=config["filter_sizes"],
            dlayers=config["dense_layers"],
            act_fn=F.relu,
            kernel_size=config["kernel_sizes"],
            padding=config["padding"],
            stride=config["stride"],
            dropout_ch=config["dropout"],
            bn=config["batchnorm"]
            )

        cv_train_acc_mean, cv_test_acc_mean, _, _ = model_trainer(
            cnn_simple, config=config,
            group=f'CNN__simple_adam_reg_{reg}', # _5f_cv, _seed_val, _hl_size{}, _lr{}
            tags=['CNN', 'adam', 'reg'], # 5fold cv, seed val, lr
            print_info=False, plot_eval=True, sound=True, 
            write_wandb=True
            )


**Verwendung von Adam für MLP**

<img src="../01_Dokumentation/wandb_images/MLP_hl2_sgd_adam.PNG" width="800" height="400">

**Verwendung von Adam für CNN**

<img src="../01_Dokumentation/wandb_images/CNN_simple_sgd_adam.PNG" width="800" height="400">


<div class="alert alert-block alert-success">

**Nutzen von Adam MLP und CNN** (ohne REG, mit SGD)  
Für das Testen von Adam wurde die Anzahl Epochen verkürzt und so das Overfitting bereits verhindert. Eine Regularisierung wird daher nur sehr gering angewendet.

**MLP**: Bei den Modellen ohne Regularisierung sind die Unterschiede zwischen SGD und Adam nicht sehr gross. Die Lernrate für Adam musste jedoch auf $lr=1^{-4}$ verringert werden, damit das Modell etwas lernte. Wenn eine Regulierung verwendet wird, findet das Training mit Adam deutlich schneller und auch stabiler statt.   

**CNN**: In beiden Fällen, mit und ohne Regularisierung, wird das Training beschleunigt. Auch die Stabilität auf dem Validation loss und Test Accuracy ist deutlich zu sehen. 

Dadurch, dass die Lernrate während dem Training angepasst werden kann, überzeugt Adam mit schnelleren Trainings und auch durch einen stabileren Verlauf des Validationloss sowie bei der Test Accuracy.
</div>

---

# Fazit und Learnings

<div class="alert alert-block alert-success">

In dieser Challenge wurden verschiedene Modellarchitekturen ausprobiert und den Einfluss der Hyperparameter untersucht. Es zeigte sich, dass die Modellarchitektur einen starken Einfluss auf die Modellperformance hat (MLP vs CNN). Die Wahl der Architektur hängt, nicht zu letzt, von der Aufgabenstellung ab und kann nicht durch ein umfassendes Hyperparameter Tunning korrigiert werden.  

Neben der Modellarchitektur sollten die Hyperparameter geziehlt optimiert werden, da der Umfang der Hyperparamter sehr schnell sehr grosse Dimensionen annehmen kann. Gute Tools wie wandb.ai können dabei helfen die Übersicht der Experimente überschaubar zu halten. Es zeigte sich auch als sinnvoll die Experimente und Resultate kurz zu beschreiben und so das weitere Vorgehen zu definieren.

Hinsichtlich Hyperparameter: Die Verwendung von Adam zeigte mehr Flexibiltät bei der Wahl der Lernrate. Sowie SGD verfügt auch Adam über weitere Parameter die noch genauer angeschaut werden sollen (Parameter zu Momentum). Mit der Regularisierung konnte gezeigt werden, dass dem Overfitting entgegen gewirkt werden kann. Neben L1 und L2 zeigte dropout und early-stopping gute Anwendungsmöglichkeiten. Die Frage welche Regularisierung in welchem Fall angewendet werden soll, ist noch offen. Kombinationen von Regularisierung scheinen denkbar aber werden schwieriger zu interpretieren. Überrascht hat die Batchnorm, diese hat das Training deutlich schneller gemacht. BN sollte jedefall als Hyperparameter im Modelltraining zur Verfügung stehen.
</div>


Ende Notebook